# ADL2CSV
ADLtagger形式のディレクトリ構造からCSVデータを抜き出し, tag情報を付加したうえでひとつのCSVファイルにまとめて出力する.

### [注意!!] タイムスタンプの桁落ち 必ず3桁にしないとsortした時変になる.

In [1]:
import os # os.mkdir(path)でdirectoryを作成

import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import datetime as dt

# Main関数
以下の２つのセルを実行

In [51]:
import os # os.mkdir(path)でdirectoryを作成

import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import datetime as dt


class ADL2Csv(object):
    """
    Generate Labeled signal data
    """
    def __init__(self, path_to_xml, path_to_csv_root_dir='./data/fromADL/', path_to_output_dir='./data/LabeledCSV/', sensor_tag='None', fix_timestamp=False):
        """
        Argument
        --------
        path_to_xml: str(Labele data file generated by ADLtagger)
        path_to_log_dir: str(path to csv files[ADLtagger format])
        path_to_output_dir: str
        """
        self.path_to_xml = path_to_xml
        self.path_to_csv_root_dir = path_to_csv_root_dir
        self.path_to_output_dir = path_to_output_dir
        self.sensor_tag = sensor_tag
        self.fix_timestamp = fix_timestamp
        
        
    """Read Data"""
    def read_data(self):
        print("Step1: Read csv files and label xml data.")
        # CSV files
        df = pd.DataFrame()
        df = self.BFS(df, self.path_to_csv_root_dir)
        #  timestampの桁落ちを修正
        print("[Read Data] fix timestamp = ", self.fix_timestamp)
        if self.fix_timestamp:
            df_tmp = df[["time"]].copy()
            df_tmp["time_1"] = "none"
            df_tmp["time_2"] = "none"
            df_tmp[["time_1","time_2"]] = df_tmp["time"].str.split(".", expand=True)
            df["time"] = df_tmp["time_1"] + "." +df_tmp["time_2"].str.zfill(3)
        self.df = df.sort_values(by=["time"]).reset_index(drop=True)
        # Label
        self.df_label = self.get_labels()
        print(">> Success: df="+str(self.df.shape)+", df_label="+str(self.df_label.shape)+"\n")
        return self.df, self.df_label
        
    ## CSVファイルを取得 ##
    def BFS(self, df, current_path):
        # 現在のディレクトリにあるディレクトリを取得
        for x in os.listdir(current_path):
            print(">>",current_path+x)
            if os.path.isdir(current_path + x):
                # ディレクトリが存在する場合 ==> より深く潜る
                df = self.BFS(df, current_path + x + '/')
            elif os.path.isfile(current_path + x):
                # ファイル場合 ==> CSVか判定 ==> YES: dfに追加
                root, ext = os.path.splitext(x)
                if ext == ".csv":
                    df = pd.concat([df, pd.read_csv(current_path+x)])
        return df

    ## XMLの解析 ##
    def get_labels(self):
        def parse_time_str(s):
            try:
                ymd, HMS, ms = s[0:10].replace('-', ''), s[11:19], s[20:]
            except IndexError as e:
                ymd, HMS, ms = s[0:10].replace('-', ''), s[11:19], '000'
            return str(ymd)+'_'+str(HMS)+'.'+str(ms).ljust(3, '0')
        # XML
        tree = ET.parse(self.path_to_xml)
        elem = tree.getroot()
        df_label = []
        for e in elem.findall("labellist"):
            label = e.find("eventtype").text
            start = parse_time_str(e.find("start").text)
            end = parse_time_str(e.find("end").text)
            df_label.append([label, start, end])
        return pd.DataFrame(df_label, columns=["label", "start", "end"])

    
    
    """Add Labels"""
    def add_label(self):
        print("Step2: Add label to sensor data.")
        df, df_label =  self.df, self.df_label
        # timedelta型に変換
        df["time"] = pd.to_datetime(df["time"], format="%Y%m%d_%H:%M:%S.%f") 
        df_label["start"] = pd.to_datetime(df_label["start"], format="%Y%m%d_%H:%M:%S.%f")
        df_label["end"] = pd.to_datetime(df_label["end"], format="%Y%m%d_%H:%M:%S.%f")
        
        labels, label_ids = ["none" for i in range(len(df)) ], [-1 for i in range(len(df)) ]
        for i in range(len(df_label)):
            label, start, end = df_label.loc[i,"label"], df_label.loc[i,"start"], df_label.loc[i,"end"]
            index = df[(df["time"] >= start) & (df["time"] <= end)].index
            # label配列を修正
            for j in index:
                labels[j], label_ids[j] = label, i
            if i%50 == 0:
                print(">> id="+str(i).zfill(4)+": label=`"+label+"`,\t\tTime=[ "+str(start)+", "+str(end)+"]")
        df["label"], df["label_id"] = labels, label_ids 
        self.df = df
        print(">> Success: df.shape=", self.df.shape, "\n")
        return df


    """CSV書き出し"""
    def to_csv(self):
        print("Step3: Write CSV files.")
        # ディレクトリを作成
        df, df_label, output_path = self.df, self.df_label, self.path_to_output_dir
        if not os.path.isdir(output_path):
            os.mkdir(output_path)
        df.to_csv(output_path+self.sensor_tag+"_labeled.csv")
        print(">> Success: Write csv.[", output_path+self.sensor_tag+"_labeled.csv", "]")
        df_label.to_csv(output_path+"label.csv")
        print(">> Success: Write csv.[", output_path+"labele.csv", "]")
        df_label[["label", "start"]].groupby(by=["label"]).count().sort_values(["start"], ascending=False).to_csv(output_path+"label_counts.csv")
    
    
# converter = ADL2Csv(path_to_xml='./data/2017_09_21/03_fromADL/data.xml',
#                     path_to_csv_root_dir='./data/2017_09_21/03_fromADL/data/acc2/',
#                     path_to_output_dir='./data/2017_09_21/04_Labeled/',
#                    sensor_tag="acc")
# df, df_label = converter.read_data()
# df = converter.add_label()
# converter.to_csv()
# display(df.head(10))
#display(df_label.head(10))

In [52]:

root_dir = "/root/upconversion/data/2017_12_15/08_higashide"


converter = ADL2Csv(path_to_xml=root_dir+"/data.xml",
                    path_to_csv_root_dir=root_dir+"/data/acc2/",
                    path_to_output_dir='./data/2017_12_15/',
                   sensor_tag="acc",
                   fix_timestamp=True)
df, df_label = converter.read_data()
df = converter.add_label()
converter.to_csv()
display(df.head(10))

Step1: Read csv files and label xml data.
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_183100_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_182500_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_182200_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_183000_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_182400_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_184000_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_183600_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_183800_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_183200_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/da

,time,x,y,z,label,label_id
0,2017-12-15 18:18:27.731,116,-1038,10193,none,-1
1,2017-12-15 18:18:27.732,65,-1026,10208,none,-1
2,2017-12-15 18:18:27.733,14,-974,10242,none,-1
3,2017-12-15 18:18:27.734,-3,-972,10262,none,-1
4,2017-12-15 18:18:27.735,12,-991,10289,none,-1
5,2017-12-15 18:18:27.736,-1,-967,10245,none,-1
6,2017-12-15 18:18:27.737,-54,-935,10245,none,-1
7,2017-12-15 18:18:27.738,-89,-940,10286,none,-1
8,2017-12-15 18:18:27.739,-89,-996,10269,none,-1
9,2017-12-15 18:18:27.740,-71,-1028,10227,none,-1


In [55]:
""" 2017.12.15
"""
import pandas as pd


root_dir = "/root/upconversion/data/2017_12_15/"
sensor = "acc"
name_list = [
    "01_kawabe", "02_higashinaka", "03_yasuda", "04_kamiya",
    "05_ishiyama", "06_yoshimura", "07_aiko", "08_higashide",
]


for _name in name_list[:]:
    print(">> ", _name)
    root_dir = "/root/upconversion/data/2017_12_15/{}".format(_name)
    df_raw = pd.read_csv(root_dir+"/data/Labeled/{}_labeled.csv".format(sensor), index_col=0)
    
    converter = ADL2Csv(path_to_xml=root_dir+"/data.xml",
                        path_to_csv_root_dir=root_dir+"/data/acc2/",
                        path_to_output_dir=root_dir+"/data/Labeled/",
                        sensor_tag=sensor,
                        fix_timestamp=True)
    df, df_label = converter.read_data()
    df = converter.add_label()
    converter.to_csv()
    display(df.head(10))

>>  01_kawabe


/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Step1: Read csv files and label xml data.
>> /root/upconversion/data/2017_12_15/01_kawabe/data/acc2/acc2_R
>> /root/upconversion/data/2017_12_15/01_kawabe/data/acc2/acc2_R/20171215_115700_acc2.csv
>> /root/upconversion/data/2017_12_15/01_kawabe/data/acc2/acc2_R/20171215_120400_acc2.csv
>> /root/upconversion/data/2017_12_15/01_kawabe/data/acc2/acc2_R/20171215_120100_acc2.csv
>> /root/upconversion/data/2017_12_15/01_kawabe/data/acc2/acc2_R/20171215_121000_acc2.csv
>> /root/upconversion/data/2017_12_15/01_kawabe/data/acc2/acc2_R/20171215_115800_acc2.csv
>> /root/upconversion/data/2017_12_15/01_kawabe/data/acc2/acc2_R/20171215_120200_acc2.csv
>> /root/upconversion/data/2017_12_15/01_kawabe/data/acc2/acc2_R/20171215_114700_acc2.csv
>> /root/upconversion/data/2017_12_15/01_kawabe/data/acc2/acc2_R/20171215_120000_acc2.csv
>> /root/upconversion/data/2017_12_15/01_kawabe/data/acc2/acc2_R/20171215_114800_acc2.csv
>> /root/upconversion/data/2017_12_15/01_kawabe/data/acc2/acc2_R/20171215_121100_ac

,time,x,y,z,label,label_id
0,2017-12-15 11:47:42.121,2152,38,10125,none,-1
1,2017-12-15 11:47:42.122,2174,67,10115,none,-1
2,2017-12-15 11:47:42.123,2198,72,10125,none,-1
3,2017-12-15 11:47:42.124,2169,-18,10125,none,-1
4,2017-12-15 11:47:42.125,2186,-38,10142,none,-1
5,2017-12-15 11:47:42.126,2176,-48,10144,none,-1
6,2017-12-15 11:47:42.127,2130,-35,10135,none,-1
7,2017-12-15 11:47:42.128,2132,-18,10186,none,-1
8,2017-12-15 11:47:42.129,2103,-28,10164,none,-1
9,2017-12-15 11:47:42.130,2066,-28,10135,none,-1


>>  02_higashinaka
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2017_12_15/02_higashinaka/data/acc2/acc2_R
>> /root/upconversion/data/2017_12_15/02_higashinaka/data/acc2/acc2_R/20171215_134600_acc2.csv
>> /root/upconversion/data/2017_12_15/02_higashinaka/data/acc2/acc2_R/20171215_135000_acc2.csv
>> /root/upconversion/data/2017_12_15/02_higashinaka/data/acc2/acc2_R/20171215_134000_acc2.csv
>> /root/upconversion/data/2017_12_15/02_higashinaka/data/acc2/acc2_R/20171215_134700_acc2.csv
>> /root/upconversion/data/2017_12_15/02_higashinaka/data/acc2/acc2_R/20171215_134200_acc2.csv
>> /root/upconversion/data/2017_12_15/02_higashinaka/data/acc2/acc2_R/20171215_134800_acc2.csv
>> /root/upconversion/data/2017_12_15/02_higashinaka/data/acc2/acc2_R/20171215_135200_acc2.csv
>> /root/upconversion/data/2017_12_15/02_higashinaka/data/acc2/acc2_R/20171215_133800_acc2.csv
>> /root/upconversion/data/2017_12_15/02_higashinaka/data/acc2/acc2_R/20171215_133600_acc2.csv
>> /root/upcon

...

,time,x,y,z,label,label_id
0,2017-12-15 16:46:10.496,-6263,3434,6888,none,-1
1,2017-12-15 16:46:10.497,-6224,3419,6912,none,-1
2,2017-12-15 16:46:10.498,-6238,3417,6922,none,-1
3,2017-12-15 16:46:10.499,-6256,3417,6893,none,-1
4,2017-12-15 16:46:10.500,-6229,3422,6895,none,-1
5,2017-12-15 16:46:10.501,-6216,3453,6954,none,-1
6,2017-12-15 16:46:10.502,-6229,3463,6963,none,-1
7,2017-12-15 16:46:10.503,-6207,3463,6907,none,-1
8,2017-12-15 16:46:10.504,-6194,3473,6878,none,-1
9,2017-12-15 16:46:10.505,-6187,3471,6866,none,-1


>>  07_aiko
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2017_12_15/07_aiko/data/acc2/acc2_R
>> /root/upconversion/data/2017_12_15/07_aiko/data/acc2/acc2_R/20171215_173600_acc2.csv
>> /root/upconversion/data/2017_12_15/07_aiko/data/acc2/acc2_R/20171215_174400_acc2.csv
>> /root/upconversion/data/2017_12_15/07_aiko/data/acc2/acc2_R/20171215_175000_acc2.csv
>> /root/upconversion/data/2017_12_15/07_aiko/data/acc2/acc2_R/20171215_174900_acc2.csv
>> /root/upconversion/data/2017_12_15/07_aiko/data/acc2/acc2_R/20171215_173700_acc2.csv
>> /root/upconversion/data/2017_12_15/07_aiko/data/acc2/acc2_R/20171215_174100_acc2.csv
>> /root/upconversion/data/2017_12_15/07_aiko/data/acc2/acc2_R/20171215_173400_acc2.csv
>> /root/upconversion/data/2017_12_15/07_aiko/data/acc2/acc2_R/20171215_173800_acc2.csv
>> /root/upconversion/data/2017_12_15/07_aiko/data/acc2/acc2_R/20171215_174700_acc2.csv
>> /root/upconversion/data/2017_12_15/07_aiko/data/acc2/acc2_R/20171215_175100_acc2.csv
>> 

,time,x,y,z,label,label_id
0,2017-12-15 17:33:07.211,467,-867,9827,none,-1
1,2017-12-15 17:33:07.212,487,-889,9871,none,-1
2,2017-12-15 17:33:07.213,448,-894,9873,none,-1
3,2017-12-15 17:33:07.214,426,-886,9820,none,-1
4,2017-12-15 17:33:07.215,457,-911,9800,none,-1
5,2017-12-15 17:33:07.216,435,-916,9805,none,-1
6,2017-12-15 17:33:07.217,433,-940,9847,none,-1
7,2017-12-15 17:33:07.218,428,-933,9859,none,-1
8,2017-12-15 17:33:07.219,445,-957,9834,none,-1
9,2017-12-15 17:33:07.220,499,-982,9805,none,-1


>>  08_higashide
Step1: Read csv files and label xml data.
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_183100_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_182500_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_182200_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_183000_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_182400_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_184000_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_183600_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_183800_acc2.csv
>> /root/upconversion/data/2017_12_15/08_higashide/data/acc2/acc2_R/20171215_183200_acc2.csv
>> /root/upconversion/data/2017_12_1

,time,x,y,z,label,label_id
0,2017-12-15 18:18:27.731,116,-1038,10193,none,-1
1,2017-12-15 18:18:27.732,65,-1026,10208,none,-1
2,2017-12-15 18:18:27.733,14,-974,10242,none,-1
3,2017-12-15 18:18:27.734,-3,-972,10262,none,-1
4,2017-12-15 18:18:27.735,12,-991,10289,none,-1
5,2017-12-15 18:18:27.736,-1,-967,10245,none,-1
6,2017-12-15 18:18:27.737,-54,-935,10245,none,-1
7,2017-12-15 18:18:27.738,-89,-940,10286,none,-1
8,2017-12-15 18:18:27.739,-89,-996,10269,none,-1
9,2017-12-15 18:18:27.740,-71,-1028,10227,none,-1
